In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import sys
import xlwt
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:windosSize-1])
        y.append(windows[i][-1:])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel():
    regressor = Sequential()
    #regressor.add(Bidirectional(LSTM(units=50,return_sequences=True),input_shape = (X_train.shape[1], 1)))
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    # Compiling
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1

##get data##

def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def train(regressor,sc, X_train, y_train, X_test, y_test,epochs):
    for i in range(epochs):
        regressor.fit(X_train, y_train,validation_split=0.2, epochs = 1, batch_size = 32,verbose=2)
        predicted = sc.inverse_transform(regressor.predict(X_test))
        originY = sc.inverse_transform (y_test)
        mse = mean_squared_error(predicted, originY)
        print("Epoch : " +str(i)+", MSE : ["+str(mse)+"]")
        print('-------------------------------------------')
  
    return mse

Using TensorFlow backend.


In [ ]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()


epochs = 200

stationList = getStationList(cursor)

col=1
for station in stationList:
    print("training : " +station)
    MSEs = []
    for windowSize in range(2,12):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        regressor = buildModel()
        mse = train(regressor,sc,X_train, y_train, X_test, y_test,epochs)
        MSEs.append(mse)
        
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,epochs,station)
    row = 1
    for m in MSEs:
        sheet1.write(row,1,m)
        row+=1
    book.save("excelFiles/LSTM/LSTMresult"+station+".xls")
        
    print('check point at ' + str(datetime.datetime.now()))

db.close()





training : 二林
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 3s - loss: 0.0024 - val_loss: 8.7616e-04
Epoch : 0, MSE : [26.83813231013203]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 2s - loss: 9.7743e-04 - val_loss: 8.7189e-04
Epoch : 1, MSE : [26.671965012772944]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.5797e-04 - val_loss: 9.2949e-04
Epoch : 2, MSE : [27.93920583856115]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.4307e-04 - val_loss: 8.7655e-04
Epoch : 3, MSE : [26.883567585456337]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.2458e-04 - val_loss: 9.2058e-04
Epoch : 4, MSE : [27.40870336493148]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples

 - 1s - loss: 9.0878e-04 - val_loss: 8.8219e-04
Epoch : 43, MSE : [27.228998151028758]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8512e-04 - val_loss: 8.9383e-04
Epoch : 44, MSE : [27.934411913430417]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9619e-04 - val_loss: 8.9027e-04
Epoch : 45, MSE : [27.827107703944968]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8731e-04 - val_loss: 8.7476e-04
Epoch : 46, MSE : [27.125089915311268]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8629e-04 - val_loss: 8.7644e-04
Epoch : 47, MSE : [27.09571444458975]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9931e-04 - val_loss: 8.7712e-04
Epo

Epoch : 86, MSE : [27.45233525488455]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9735e-04 - val_loss: 8.7946e-04
Epoch : 87, MSE : [27.57684879801465]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.0426e-04 - val_loss: 8.9672e-04
Epoch : 88, MSE : [27.17315484504364]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9104e-04 - val_loss: 9.0082e-04
Epoch : 89, MSE : [27.994137246284822]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.0088e-04 - val_loss: 9.9651e-04
Epoch : 90, MSE : [30.632478831175035]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8321e-04 - val_loss: 8.9629e-04
Epoch : 91, MSE : [27.769583030689894]
--------------

 - 1s - loss: 9.0474e-04 - val_loss: 8.8778e-04
Epoch : 130, MSE : [27.897845324158006]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8908e-04 - val_loss: 8.8270e-04
Epoch : 131, MSE : [27.533668986737354]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8902e-04 - val_loss: 8.9176e-04
Epoch : 132, MSE : [27.78603128070553]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8734e-04 - val_loss: 8.8383e-04
Epoch : 133, MSE : [27.371967308365807]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9692e-04 - val_loss: 8.7828e-04
Epoch : 134, MSE : [27.269064461199157]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 9.0230e-04 - val_loss: 9.2676e-0

 - 1s - loss: 8.8257e-04 - val_loss: 8.8634e-04
Epoch : 173, MSE : [27.815688496978648]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9961e-04 - val_loss: 8.8416e-04
Epoch : 174, MSE : [27.936128141382436]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9184e-04 - val_loss: 8.8032e-04
Epoch : 175, MSE : [27.223766221146]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9430e-04 - val_loss: 8.8966e-04
Epoch : 176, MSE : [27.50613912505511]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.9939e-04 - val_loss: 8.9063e-04
Epoch : 177, MSE : [27.87939762373808]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.8720e-04 - val_loss: 9.0184e-04
Ep

...

 - 6s - loss: 0.0012 - val_loss: 9.8909e-04
Epoch : 140, MSE : [29.97197081521358]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 0.0010
Epoch : 141, MSE : [30.233635507618803]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 9.8126e-04
Epoch : 142, MSE : [30.147689543148974]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 0.0010
Epoch : 143, MSE : [29.64391871732158]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 9.7550e-04
Epoch : 144, MSE : [29.016928968824192]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 0.0010
Epoch : 145, MSE : [29.324665085402

Epoch : 184, MSE : [29.953340878116148]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 9.7377e-04
Epoch : 185, MSE : [30.341632119237506]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 9.6934e-04
Epoch : 186, MSE : [29.7489846562822]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 9.9489e-04
Epoch : 187, MSE : [29.695721029975044]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 9.7018e-04
Epoch : 188, MSE : [30.974085680346413]
-------------------------------------------
Train on 16102 samples, validate on 4026 samples
Epoch 1/1
 - 6s - loss: 0.0012 - val_loss: 0.0010
Epoch : 189, MSE : [30.427405443708185]
-------------------------------

Epoch : 29, MSE : [29.541639522691252]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 8s - loss: 0.0013 - val_loss: 0.0013
Epoch : 30, MSE : [28.571520957718405]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0013
Epoch : 31, MSE : [28.11661366877948]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0013
Epoch : 32, MSE : [29.434865830957822]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0013
Epoch : 33, MSE : [29.452148655402425]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 34, MSE : [28.707167038742607]
-------------------------------------------
Train on

 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 75, MSE : [28.975955059028795]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0013
Epoch : 76, MSE : [29.050805691122488]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 77, MSE : [28.466870590636614]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 8s - loss: 0.0012 - val_loss: 0.0013
Epoch : 78, MSE : [29.335352440282236]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 79, MSE : [28.50514302351518]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0013
Epoch : 80, MSE : [29.35434871901673]
-------------

 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 120, MSE : [29.46677503921176]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 121, MSE : [29.53900834681274]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 122, MSE : [29.7390566431889]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 123, MSE : [30.127561734805777]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 124, MSE : [30.03337946147459]
-------------------------------------------
Train on 16101 samples, validate on 4026 samples
Epoch 1/1
 - 7s - loss: 0.0012 - val_loss: 0.0012
Epoch : 125, MSE : [29.951632588039956]
----------

In [25]:


book.save("Test1EXCCCC.xls")